In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import make_scorer
from sklearn.model_selection import TimeSeriesSplit

# RMSPE Funktion definieren
def rmspe(y_true, y_pred):
    y_pred = np.array(y_pred)  # Umwandlung der Vorhersagen in ein Numpy-Array
    mask = y_true != 0
    y_true_filtered = y_true[mask]
    y_pred_filtered = y_pred[mask]
    return np.sqrt(np.mean(((y_true_filtered - y_pred_filtered) / y_true_filtered) ** 2))

# RMSPE als Scorer definieren
rmspe_scorer = make_scorer(rmspe, greater_is_better=False)

# Laden des Datensatzes
data_cleaned = "../data/cleaned_train_alt.csv"
data = pd.read_csv(data_cleaned, delimiter=",", encoding="latin", header=0, thousands=",", decimal='.', low_memory=False)

# Zielvariable und Features definieren
X = data.drop('Sales', axis=1)
y = data['Sales']

# Definiere die numerischen und kategorischen Features
numerical_features = ['year', 'month', 'day', 'week_of_year', 'lag_1', 'lag_7']
already_encoded_features = ['Open', 'Promo', 'promo2']
categorical_features_to_encode = ['Store', 'DayOfWeek', 'StoreType', 'StateHoliday', 'Assortment']

# Preprocessing Pipeline für numerische und kategorische Features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('enc', 'passthrough', already_encoded_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_to_encode)
    ])

# Modell für RandomForest erstellen
model = RandomForestRegressor(n_estimators=1, random_state=42)

# Pipeline erstellen
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

#Funktion zur Vorhersage der nächsten x Tage (wie im Originalcode)
def predict_multiple_days_with_updates(X_last, test_data, num_days):
    predictions = []
    X_next = X_last.copy()
    X_next = pd.DataFrame([X_next])  # In DataFrame umwandeln
    last_lags = list(X_last[['lag_1']].values.flatten())  # Initialisiere letzte Lags

    for day in range(num_days):
        # Preprocess die Daten vor der Vorhersage
        X_next_preprocessed = pipeline.named_steps['preprocessor'].transform(X_next)
        
        # Vorhersage des nächsten Tages
        y_next_pred = pipeline.named_steps['model'].predict(X_next_preprocessed)[0]
        predictions.append(y_next_pred)

        # Aktualisiere die Lag-Features basierend auf der Vorhersage
        last_lags.append(y_next_pred)
        last_lags.pop(0)  # Aktualisiere Lag-Werte

        X_next.loc[:, 'lag_1'] = y_next_pred
        X_next.loc[:, 'lag_7'] = last_lags[0]

        # Aktualisiere andere Features (Tag, Monat, Woche)
        X_next.loc[:, 'day'] += 1
        if X_next.loc[:, 'day'].values[0] > 31:
            X_next.loc[:, 'day'] = 1
            X_next.loc[:, 'month'] += 1
        if X_next.loc[:, 'month'].values[0] > 12:
            X_next.loc[:, 'month'] = 1
            X_next.loc[:, 'year'] += 1

        X_next.loc[:, 'week_of_year'] = (X_next['week_of_year'] % 52) + 1
        X_next.loc[:, 'DayOfWeek'] = (X_next['DayOfWeek'] % 7) + 1

        # Promo- und andere Features aus Testdaten übernehmen
        if day < len(test_data):
            X_next.loc[:, 'Promo'] = test_data.iloc[day]['Promo']
            X_next.loc[:, 'Open'] = test_data.iloc[day]['Open']
            X_next.loc[:, 'StateHoliday'] = test_data.iloc[day]['StateHoliday']

    return predictions

# TimeSeriesSplit für Cross-Validation
tscv = TimeSeriesSplit(n_splits=5)

# Überprüfung mit Cross-Validation
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

cv_rmspe_scores = []
for train_idx, test_idx in tscv.split(X_train):
    X_train_cv, X_test_cv = X_train.iloc[train_idx], X_train.iloc[test_idx]
    y_train_cv, y_test_cv = y_train.iloc[train_idx], y_train.iloc[test_idx]

    # Pipeline auf den Cross-Validation-Trainingsdaten fitten
    pipeline.fit(X_train_cv, y_train_cv)

    # Lag-Feature-Vorhersage für die CV-Testdaten
    X_last_cv = X_test_cv.iloc[0]  # Starte mit dem ersten Testtag
    num_days_cv = len(X_test_cv)    # Vorhersage für alle Testtage
    future_predictions_cv = predict_multiple_days_with_updates(X_last_cv, X_test_cv, num_days_cv)

    # Berechne RMSPE für diesen Split
    rmspe_score = rmspe(y_test_cv.values, future_predictions_cv)
    cv_rmspe_scores.append(rmspe_score)

# Ausgabe der Ergebnisse
mean_cv_rmspe = np.mean(cv_rmspe_scores)
print(f"Durchschnittliche RMSPE (Cross-Validation): {mean_cv_rmspe:.4f}")

# Nimm die letzten Daten als Startpunkt für die Vorhersage
X_last = X_test.iloc[-60]

# Vorhersage für die nächsten 7 Tage unter Verwendung von Promo-Informationen aus dem Testdatensatz
num_days = 60
test_data_next_days = X_test.iloc[:num_days]  # Extrahiere die Testdaten für die nächsten Tage

# Vorhersagen für die nächsten Tage machen
future_predictions = predict_multiple_days_with_updates(X_last, test_data_next_days, num_days)

# Ergebnisse anzeigen
print(f"Vorhersagen für die nächsten {num_days} Tage:", future_predictions)

# Tatsächliche Verkaufszahlen für die nächsten 7 Tage aus dem Testdatensatz extrahieren
actual_sales_next_days = y_test.iloc[:num_days].values

# Berechnung der RMSPE für die Vorhersagen
prediction_rmspe = rmspe(actual_sales_next_days, future_predictions)

# Ergebnis anzeigen
print(f"RMSPE der Vorhersagen für die nächsten {num_days} Tage: {prediction_rmspe:.4f}")